In [ ]:
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_excel.html
https://docs.sqlalchemy.org/en/latest/errors.html#error-e3q8
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html 

In [8]:
#volcanoes = open('/home/d.hayes/Problem_Set_4/GVP_Eruption_Results.xls')

In [40]:
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String, Float, MetaData, ForeignKey
from sqlalchemy import exists
from sqlalchemy import sql, select, join, desc

# Create a sqlite database 
engine = create_engine('sqlite:///volcanoes.sqlite')

metadata=MetaData()

#Making the Volcanoes table
try:
    Volcanoes=Table('Volcanoes', metadata, autoload=True)
except:
    Volcanoes = Table ('Volcanoes', metadata,
                    Column('Volcano_Number', Integer),
                    Column('Volcano_Name', String, primary_key = True),
                    Column('Latitude', String),
                    Column('Longitude', String)
                   )

try:
    Eruptions=Table('Eruptions', metadata, autoload=True)
except:
    Eruptions = Table ('Eruptions', metadata,
                     Column('Eruption_Number', Integer, primary_key = True),
                     Column('Volcano_Name', String, ForeignKey('Volcanoes.Volcano_Name')),
                     Column('Eruption_Category', String),
                     Column('Area_of_Activity', String),
                     Column('VEI', String),
                     Column('Start_Year', Integer),
                     Column('Start_Month', Integer),
                     Column('Start_Day', Integer),
                     Column('Evidence_Method', String),
                     Column('End_Year', Integer),
                     Column('End_Month', Integer),
                     Column('End_Day', Integer)
                    )
                 
metadata.create_all(engine)

In [13]:
#df=pd.read_excel("GVP_Eruption_Results.xls.xlsx")

import pandas as pd

df = pd.ExcelFile('GVP_Eruption_Results.xls.xlsx').parse('Eruption List');
#Check to make sure df worked:
#print(df.head())

#conn = engine.connect()
#df.to_sql('Volcanoes', conn, if_exists='replace', chunksize=50) 
#df.to_sql(volcanoes, con=engine, index=True, index_label=None, chunksize=None, dtype={"Volcano Number": Integer(),"Volcano Name": String(), "Eruption Number": Integer, "Latitude": Float, "Longitude": Float})

#volcanoes = open('/home/d.hayes/Problem_Set_4/GVP_Eruption_Results.xls')


In [14]:
df.head()
df.shape

(11135, 24)

In [42]:
reader = df.to_dict()
Volcano_dict={}
print(reader["Volcano Name"][212])
# Read through the file and make a dictionary for airport codes.
# This gets a unique list of airport codes.
for Line in range(11135):
    if reader['Volcano Name'][Line] not in Volcano_dict:
        Volcano_dict[reader['Volcano Name'][Line]]=reader['Volcano Number'][Line], reader["Longitude"][Line], reader['Latitude'][Line]  
print(len(Volcano_dict))
print (Volcano_dict)

Karkar
920
{'Ulawun': (252120, 151.33, -5.05), 'Ketoi': (290200, 152.475, 47.35), 'Sarychev Peak': (290240, 153.2, 48.092), 'Semisopochnoi': (311060, 179.58, 51.93), 'Veniaminof': (312070, -159.38, 56.17), 'Alaid': (290390, 155.565, 50.861), 'Sangay': (352090, -78.341, -2.005), 'Rincon de la Vieja': (345020, -85.324, 10.83), 'Nishinoshima': (284096, 140.874, 27.247), 'Negra, Sierra': (353050, -91.17, -0.83), 'Telica': (344040, -86.84, 12.606), 'Krakatau': (262000, 105.423, -6.102), 'Fernandina': (353010, -91.55, -0.37), 'Great Sitkin': (311120, -176.13, 52.076), 'Kerinci': (261170, 101.264, -1.697), 'Merapi': (263250, 110.446, -7.54), 'Karymsky': (300130, 159.443, 54.049), 'Marapi': (261140, 100.474, -0.38), 'San Cristobal': (344020, -87.004, 12.702), 'Fournaise, Piton de la': (233020, 55.708, -21.244), 'Dieng Volcanic Complex': (263200, 109.879, -7.2), 'Kirishimayama': (282090, 130.862, 31.934), 'Karangetang': (267020, 125.407, 2.781), 'Kusatsu-Shiranesan': (283120, 138.528, 36.618), 

In [43]:
# Add the Volcano_dict codes to the Volcanoes table
from sqlalchemy.orm import sessionmaker
conn = engine.connect()

def insert_Volcanoes(VolName, VolNumber, Long, Lat):
    ins=Volcanoes.insert().values(Volcano_Name=VolName,
                                  Volcano_Number=VolNumber,
                                  Longitude=Long,
                                  Latitude=Lat)
    result = conn.execute(ins)

for key, value in Volcano_dict.items(): 
    insert_Volcanoes(key, value[0], value[1], value[2])

In [50]:
#Add the data to the Eruptions table
#It worked!!

df.to_sql('Eruptions', conn, if_exists='replace')  